<a href="https://colab.research.google.com/github/esassoc/qanat-community/blob/develop/Qanat.CommunityAPI/Examples/Hackathon_2025_ClaimedParcelsExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# Download API documentation
import requests

try:
    url = "https://raw.githubusercontent.com/esassoc/qanat-community/develop/Qanat.CommunityAPI/Examples/groundwater-accounting-platform-api.json"
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes

    with open("groundwater-accounting-platform-api.json", "w") as f:
        f.write(response.text)

    # Load API documentation
    import json

    with open('groundwater-accounting-platform-api.json', 'r') as f:
        api_documentation = json.load(f)

    # API key setup
    from google.colab import userdata

    api_key = None  # Initialize api_key to None

    try:
      api_key = userdata.get('API_KEY')
      if not api_key:
          # If the secret exists but is empty, print message and continue to check api_key
          print("API key found in Colab Secrets but is empty. Please provide your API key.")
          api_key = None # Ensure api_key is None if empty

    except userdata.SecretNotFoundError:
      # If the secret does not exist, print instructions and continue to check api_key
      print("-----------------------------------------------------------------------")
      print("API key not found in Colab Secrets.")
      print("Please add your API key to Colab Secrets:")
      print("1. Click on the '🔑' icon in the left sidebar.")
      print("2. Click on 'New secret'.")
      print("3. For 'Name', enter 'API_KEY'.")
      print("4. For 'Value', paste your API key.")
      print("5. Click 'Save secret'.")
      print("Then, run this cell again.")
      print("-----------------------------------------------------------------------")
      api_key = None # Ensure api_key is None if not found

    # Only proceed with API call if API key is available
    if api_key:
        print("API key successfully loaded from Colab Secrets.")

        # Initial API call to list geographies
        base_url = api_documentation['servers'][0]['url'].rstrip('/')
        geographies_path = '/geographies'

        geographies_url = f"{base_url}{geographies_path}"

        headers = {
            "x-api-key": api_key
        }

        print(f"Attempting to call: {geographies_url}")
        response = requests.get(geographies_url, headers=headers)

        if response.status_code == 200:
          geographies_data = response.json()
          print("Available Geographies:")
          print(json.dumps(geographies_data, indent=2))
        else:
          print(f"Error: API call failed with status code {response.status_code}")
          print(response.text)
          if response.status_code == 401:
              print("Authentication Error: Please check your API key in Colab secrets ('API_KEY').")

except requests.exceptions.RequestException as e:
  print(f"Error: An error occurred during the API request: {e}")

API key successfully loaded from Colab Secrets.
Attempting to call: http://api-qa.groundwateraccounting.org/geographies
Available Geographies:
[
  {
    "GeographyID": 1,
    "GeographyName": "MIUGSA",
    "GeographyDisplayName": "Merced Irrigation-Urban Groundwater Sustainability Agency"
  },
  {
    "GeographyID": 2,
    "GeographyName": "Pajaro",
    "GeographyDisplayName": "Pajaro Valley Water Management Agency"
  },
  {
    "GeographyID": 3,
    "GeographyName": "RRB",
    "GeographyDisplayName": "Rosedale-Rio Bravo Water Storage District"
  },
  {
    "GeographyID": 4,
    "GeographyName": "Yolo",
    "GeographyDisplayName": "Yolo County Flood Control & Water Conservation District"
  },
  {
    "GeographyID": 5,
    "GeographyName": "Demo",
    "GeographyDisplayName": "Demo Geography"
  },
  {
    "GeographyID": 6,
    "GeographyName": "MSGSA",
    "GeographyDisplayName": "Merced Subbasin Groundwater Sustainability Agency"
  },
  {
    "GeographyID": 7,
    "GeographyName": "ETSG

# Task
Calculate the percentage of parcels with claimed water accounts across all geographies, using `WaterAccountPINLastUsed` as an indicator for claimed accounts.

## Get all geographies

### Subtask:
Retrieve the list of all available geographies from the API.


**Reasoning**:
Construct the URL, make the API request to get all geographies, and handle the response based on the status code.



In [59]:
geographies_url = f"{base_url}{geographies_path}"

response = requests.get(geographies_url, headers=headers)

if response.status_code == 200:
  all_geographies_data = response.json()
  print("Successfully retrieved all geographies.")
else:
  print(f"Error: API call for all geographies failed with status code {response.status_code}")
  print(response.text)

Successfully retrieved all geographies.


## Iterate through geographies

### Subtask:
For each geography, retrieve all water accounts and all parcels.


**Reasoning**:
Iterate through the geographies and retrieve all water accounts and parcels for each, storing them in dictionaries.



In [60]:
all_geography_water_accounts = {}
all_geography_parcels = {}

for geography in all_geographies_data:
    geography_id = geography['GeographyID']
    geography_name = geography['GeographyName']
    print(f"Retrieving data for Geography: {geography_name} (ID: {geography_id})")

    # Retrieve water accounts for the current geography
    water_accounts_path = api_documentation['paths']['/geographies/{geographyID}/water-accounts']['get']['parameters'][0]['in'] # This path needs to be adjusted based on the actual structure
    # Corrected path to get water accounts endpoint
    water_accounts_endpoint = api_documentation['paths']['/geographies/{geographyID}/water-accounts']['get']['summary']
    water_accounts_url = f"{base_url}/geographies/{geography_id}/water-accounts"

    print(f"  Attempting to call: {water_accounts_url}")
    response = requests.get(water_accounts_url, headers=headers)

    if response.status_code == 200:
        water_accounts_data = response.json()
        all_geography_water_accounts[geography_id] = water_accounts_data
        print(f"  Successfully retrieved {len(water_accounts_data)} water accounts for {geography_name}")
    else:
        print(f"  Error retrieving water accounts for {geography_name}: Status code {response.status_code}")
        print(response.text)
        all_geography_water_accounts[geography_id] = [] # Store empty list if retrieval fails

    # Retrieve parcels for the current geography
    parcels_endpoint = api_documentation['paths']['/geographies/{geographyID}/parcels']['get']['summary']
    parcels_url = f"{base_url}/geographies/{geography_id}/parcels"

    print(f"  Attempting to call: {parcels_url}")
    response = requests.get(parcels_url, headers=headers)

    if response.status_code == 200:
        parcels_data = response.json()
        all_geography_parcels[geography_id] = parcels_data
        print(f"  Successfully retrieved {len(parcels_data)} parcels for {geography_name}")
    else:
        print(f"  Error retrieving parcels for {geography_name}: Status code {response.status_code}")
        print(response.text)
        all_geography_parcels[geography_id] = [] # Store empty list if retrieval fails

print("Finished retrieving water accounts and parcels for all geographies.")

Retrieving data for Geography: MIUGSA (ID: 1)
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/1/water-accounts
  Successfully retrieved 2706 water accounts for MIUGSA
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/1/parcels
  Successfully retrieved 90272 parcels for MIUGSA
Retrieving data for Geography: Pajaro (ID: 2)
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/2/water-accounts
  Successfully retrieved 0 water accounts for Pajaro
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/2/parcels
  Successfully retrieved 0 parcels for Pajaro
Retrieving data for Geography: RRB (ID: 3)
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/3/water-accounts
  Successfully retrieved 124 water accounts for RRB
  Attempting to call: http://api-qa.groundwateraccounting.org/geographies/3/parcels
  Successfully retrieved 1492 parcels for RRB
Retrieving data for Geography: Yolo (

## Identify claimed water accounts

### Subtask:
For each geography, filter the water accounts to identify those that have been "claimed" based on `WaterAccountPINLastUsed`.


**Reasoning**:
Initialize a dictionary to store claimed water accounts by geography and then iterate through each geography and its water accounts to identify and store the claimed ones based on the 'WaterAccountPINLastUsed' field.



In [61]:
claimed_water_accounts_by_geography = {}

for geography_id, water_accounts_list in all_geography_water_accounts.items():
    claimed_accounts_list = []
    for account in water_accounts_list:
        if account.get('WaterAccountPINLastUsed') is not None:
            claimed_accounts_list.append(account)
    claimed_water_accounts_by_geography[geography_id] = claimed_accounts_list

print("Finished identifying claimed water accounts for all geographies.")

Finished identifying claimed water accounts for all geographies.


## Link parcels to claimed accounts

### Subtask:
For each geography, link parcels to water accounts to determine which parcels are associated with the claimed accounts.


**Reasoning**:
Iterate through each geography, create a set of claimed water account IDs, and then iterate through the parcels in that geography to find those linked to claimed accounts.



In [62]:
parcels_linked_to_claimed_accounts_by_geography = {}

for geography_id, claimed_water_accounts in claimed_water_accounts_by_geography.items():
    # Create a set of claimed water account IDs for efficient lookup
    claimed_account_ids = {account['WaterAccountID'] for account in claimed_water_accounts}

    linked_parcels_list = []
    # Get all parcels for the current geography
    all_parcels = all_geography_parcels.get(geography_id, []) # Use .get to handle potential missing geography_id

    for parcel in all_parcels:
        # Check if the parcel has a WaterAccountID and if it's in the set of claimed account IDs
        if 'WaterAccountID' in parcel and parcel['WaterAccountID'] in claimed_account_ids:
            linked_parcels_list.append(parcel)

    parcels_linked_to_claimed_accounts_by_geography[geography_id] = linked_parcels_list

print("Finished linking parcels to claimed water accounts for each geography.")

Finished linking parcels to claimed water accounts for each geography.


## Calculate percentages

### Subtask:
For each geography, calculate the percentage of parcels linked to claimed accounts out of the total parcels in that geography.


**Reasoning**:
Calculate the percentage of parcels linked to claimed accounts for each geography and store the results in a dictionary.



In [63]:
percentage_parcels_linked_to_claimed_accounts_by_geography = {}

for geography_id in all_geography_parcels.keys():
    total_parcels = len(all_geography_parcels.get(geography_id, []))
    linked_parcels = len(parcels_linked_to_claimed_accounts_by_geography.get(geography_id, []))

    if total_parcels > 0:
        percentage = (linked_parcels / total_parcels) * 100
    else:
        percentage = 0.0

    percentage_parcels_linked_to_claimed_accounts_by_geography[geography_id] = percentage

print("Percentage calculation for parcels linked to claimed accounts for each geography is complete.")

Percentage calculation for parcels linked to claimed accounts for each geography is complete.


## Calculate overall percentage

### Subtask:
Calculate the overall percentage of parcels linked to claimed accounts across all geographies.


**Reasoning**:
Calculate the overall percentage of parcels linked to claimed accounts across all geographies by summing the total parcels and linked parcels from all geographies and then computing the percentage.



In [64]:
total_all_parcels = 0
for parcels_list in all_geography_parcels.values():
    total_all_parcels += len(parcels_list)

total_parcels_linked_to_claimed_accounts = 0
for linked_parcels_list in parcels_linked_to_claimed_accounts_by_geography.values():
    total_parcels_linked_to_claimed_accounts += len(linked_parcels_list)

if total_all_parcels > 0:
    overall_percentage_parcels_linked_to_claimed_accounts = (total_parcels_linked_to_claimed_accounts / total_all_parcels) * 100
else:
    overall_percentage_parcels_linked_to_claimed_accounts = 0.0

print(f"Overall percentage of parcels linked to claimed accounts: {overall_percentage_parcels_linked_to_claimed_accounts:.2f}%")

Overall percentage of parcels linked to claimed accounts: 3.10%


## Present results

### Subtask:
Display the calculated percentages for each geography and the overall percentage.


**Reasoning**:
Display the calculated percentages for each geography and the overall percentage as requested by the subtask instructions.



In [65]:
print("Percentage of parcels linked to claimed accounts by Geography:")
for geography_id, percentage in percentage_parcels_linked_to_claimed_accounts_by_geography.items():
    print(f"Geography ID {geography_id}: {percentage:.2f}%")

print("\nOverall Percentage of parcels linked to claimed accounts:")
print(f"{overall_percentage_parcels_linked_to_claimed_accounts:.2f}%")

Percentage of parcels linked to claimed accounts by Geography:
Geography ID 1: 2.31%
Geography ID 2: 0.00%
Geography ID 3: 0.00%
Geography ID 4: 0.00%
Geography ID 5: 34.92%
Geography ID 6: 36.01%
Geography ID 7: 34.92%
Geography ID 8: 0.03%
Geography ID 9: 1.21%
Geography ID 10: 1.74%

Overall Percentage of parcels linked to claimed accounts:
3.10%


## Summary:

### Data Analysis Key Findings

*   The overall percentage of parcels linked to claimed water accounts across all geographies is 3.10%.
*   The percentage of parcels linked to claimed accounts varies by geography, as shown in the presented results.

### Insights or Next Steps

*   Further analysis could investigate the factors contributing to the variation in claimed water account percentages across different geographies.
*   Explore if there are any correlations between the percentage of claimed water accounts and other geographical or socioeconomic data.
